In [1]:
import numpy as np
from sklearn.decomposition import PCA 
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

In [2]:
# Função para carregar o conjunto de dados MNIST
def load_mnist(image_file, label_file):
    with open(label_file, 'rb') as lbpath:
        lbpath.read(8)
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8)

    with open(image_file, 'rb') as imgpath:
        imgpath.read(16)
        images = np.frombuffer(imgpath.read(), dtype=np.uint8).reshape(len(labels), 784)

    return images, labels

In [3]:
# one hot
def one_hot_convert(vec):
    matrix = []
    for idx in vec:
      m = np.zeros((10, 1))
      m[idx] = 1
      matrix.append(m)
    return np.array(matrix)

In [4]:
# LEITURA DOS DADOS
train_images, train_labels = load_mnist('dataset/train-images.idx3-ubyte', 'dataset/train-labels.idx1-ubyte')
test_images, test_labels = load_mnist('dataset/t10k-images.idx3-ubyte', 'dataset/t10k-labels.idx1-ubyte')

print('ORIGINAL')
print('train_images',train_images.shape)
print('train_labels',train_labels.shape)
print('test_images',test_images.shape)
print('test_labels',test_labels.shape)

ORIGINAL
train_images (60000, 784)
train_labels (60000,)
test_images (10000, 784)
test_labels (10000,)


In [5]:
# Transformar as imagens em vetores e normalizá-las
X_train = train_images.reshape(train_images.shape[0], -1)
X_test = test_images.reshape(test_images.shape[0], -1)

# Adicionar uma coluna de 1s para considerar o termo de bias (intercept) no modelo linear
X_train = np.column_stack((X_train, np.ones(X_train.shape[0])))
X_test = np.column_stack((X_test, np.ones(X_test.shape[0])))

y_train = one_hot_convert(train_labels).reshape(train_labels.shape[0], -1)
y_test = test_labels.reshape(test_labels.shape[0], -1)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(60000, 785) (60000, 10)
(10000, 785) (10000, 1)


In [6]:
train_images = train_images.reshape(train_images.shape[0], -1) / 255  # Redimensionando e normalizando as imagens de treino
test_images = test_images.reshape(test_images.shape[0], -1) / 255  # Redimensionando e normalizando as imagens de teste

# Aplicando PCA
pca = PCA(n_components=100)  # Instanciando o PCA. Aqui, escolhemos manter 100 componentes principais
train_images = pca.fit_transform(train_images)  # Aplicando o PCA nos dados de treino
test_images = pca.transform(test_images)  # Aplicando a transformação PCA nos dados de teste

# Convertendo rótulos para representação one-hot
train_labels = to_categorical(train_labels)
test_labels_oneh = to_categorical(test_labels)

In [7]:
# Criando o modelo de Perceptron Logístico
# Este é um classificador logístico, que é uma rede neural de uma única camada com função de ativação sigmoide
model = tf.keras.models.Sequential([tf.keras.layers.Dense(10, input_dim=train_images.shape[1], activation='sigmoid')])

# Definindo o otimizador SGD
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

# Compilando o modelo
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Treinando o modelo
model.fit(train_images, train_labels, epochs=300, batch_size=32, shuffle=True)

# Fazendo previsões
y_pred = model.predict(test_images)
y_pred = np.argmax(y_pred, axis=1)

# Avaliando o modelo
accuracy = accuracy_score(test_labels, y_pred)
print('Accuracy:', accuracy * 100)

Epoch 1/300
1875/1875 [==============================] - 2s 712us/step - loss: 0.4263 - accuracy: 0.8790
Epoch 2/300
1875/1875 [==============================] - 1s 700us/step - loss: 0.3239 - accuracy: 0.9081
Epoch 3/300
1875/1875 [==============================] - 1s 698us/step - loss: 0.3093 - accuracy: 0.9119
Epoch 4/300
1875/1875 [==============================] - 1s 695us/step - loss: 0.3029 - accuracy: 0.9139
Epoch 5/300
1875/1875 [==============================] - 1s 698us/step - loss: 0.2987 - accuracy: 0.9153
Epoch 6/300
1875/1875 [==============================] - 1s 718us/step - loss: 0.2965 - accuracy: 0.9160
Epoch 7/300
1875/1875 [==============================] - 1s 700us/step - loss: 0.2941 - accuracy: 0.9168
Epoch 8/300
1875/1875 [==============================] - 1s 704us/step - loss: 0.2928 - accuracy: 0.9177
Epoch 9/300
1875/1875 [==============================] - 1s 696us/step - loss: 0.2918 - accuracy: 0.9174
Epoch 10/300
1875/1875 [==============================]